In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from collections import OrderedDict
from pypfopt.efficient_frontier import EfficientFrontier
import yfinance as yf


Data 

In [2]:
# Load data
prices_df = pd.read_csv('data\Russell3000_prices_clean.csv')
prices_df['Date'] = pd.to_datetime(prices_df['Date'], dayfirst=True)
prices_df.sort_values(by='Date', inplace=True)

all_stocks = prices_df.columns[1:10]
training_stocks = all_stocks[1:7]
other_stocks = all_stocks[7:]

# Calculate returns
returns_df = prices_df[all_stocks].pct_change()
returns_df = returns_df.dropna().reset_index(drop=True)
returns_df.insert(0, 'Date', prices_df['Date'][:-1])

# Convert to numpy array (for easier calculations)
returns_matrix = returns_df.iloc[:, 1:].to_numpy()

del prices_df

In [3]:
forecast_horizon = 21 # trading days
test_start_date = pd.to_datetime('2015-01-01')
test_end_date = pd.to_datetime('2020-01-01')

messages = {"test_start": [], "test_end": []}

# Check if test_start_date exists in the DataFrame, if not, take the next date that exists
if test_start_date not in returns_df['Date'].values:
    test_start_date = returns_df['Date'][returns_df['Date'] > test_start_date].bfill().iloc[0]
    messages["test_start"] = messages["test_start"] or ['Start date not found, using closest next date instead.']

# Check if test_end_date exists in the DataFrame, if not, take the previous date that exists
if test_end_date not in returns_df['Date'].values:
    test_end_date = returns_df['Date'][returns_df['Date'] < test_end_date].ffill().iloc[-1]
    messages["test_end"] = messages["test_end"] or ['End date not found, using closest date instead.']

test_start = returns_df.loc[returns_df['Date'] == test_start_date].index[0]
test_end = returns_df.loc[returns_df['Date'] == test_end_date].index[0]

del test_start_date, test_end_date, messages

In [4]:
global train_start, train_end

methods = defaultdict(lambda: defaultdict(list)) 

train_start = (test_start - forecast_horizon) % forecast_horizon + forecast_horizon + 126
train_end = test_start # Training end is the start of the test period

x_label = ['std 1w', 'std 1m', 'std 3m', 'std 6m', 'mean_ret 1w', 'mean_ret 1m', 'mean_ret 3m', 'mean_ret 6m', 'mean_vix 1w', 'mean_vix 1m', 'mean_vix 3m', 'mean_vix 6m','beta 1w', 'beta 1m', 'beta 3m', 'beta 6m'] 
windows = [5, 21, 63, 126]

In [5]:
#get data for vix index
vix = yf.Ticker("^VIX")

vix_data = vix.history(start="2000-01-01", end="2020-01-01")
vix_data.index = vix_data.index.tz_localize(None)

returns_temp = returns_df.copy()

returns_temp['Date'] = pd.to_datetime(returns_temp['Date'])
returns_temp.set_index('Date', inplace=True)

vix_data = vix_data[["Close"]]

vix_data = vix_data.reindex(returns_temp.index)
vix_data.ffill(inplace=True)

del returns_temp, vix

Helper functions

In [33]:
#functions
def get_portfolio_weights(volatility):
    inv_volatilities = np.array([1/volatility[stock] \
                                 if volatility[stock] > 0 else 0 for stock in all_stocks])
    total = np.sum(inv_volatilities)
    return inv_volatilities / total if total > 0 else np.zeros(len(all_stocks))



def gather_portfolio_metrics(method: str, i: int, weights: OrderedDict, volatility: dict) -> None:
    # Calculate portfolio returns for each day in the window
    for stock in all_stocks:
        methods[method][stock + " predicted volatility"].append(volatility[stock])

    returns = np.dot(returns_matrix[i:i+forecast_horizon], np.array(list(weights.values())))
    methods[method]["returns"].extend(returns)
    methods[method]["std"].append(np.std(returns))



def reset_portfolio_metrics(method):
    for stock in all_stocks:
        methods[method][stock + " predicted volatility"] = []
    methods[method]["returns"] = []



def standardize_returns(returns_matrix):
    mean_returns = np.mean(returns_matrix)
    std_returns = np.std(returns_matrix)
    standardized_returns = np.where(std_returns!=0, (returns_matrix - mean_returns) / std_returns, 0)
    return standardized_returns.reshape(-1, 1)



def get_linear_regression_model(X, y, fit_intercept=True, positive=False):
    model = LinearRegression(fit_intercept=fit_intercept, positive=positive)
    model.fit(X, y)

    return model



def predict_corr(betas: list) -> np.ndarray:
    predicted_correlations = np.outer(betas, betas)
    psi_matrix = np.diag(1 - np.diag(predicted_correlations))
    return predicted_correlations + psi_matrix


#bete samo za test, netreba za 
def get_betas():
    betas_corr = {}

    for i in range(test_start, test_end, forecast_horizon):

        temp = []
        for stock in all_stocks:
            window = 126

            standardized_returns = standardize_returns(returns_df[stock][i - window:i])
            standardized_market_index_returns = standardize_returns(market_index_returns[i - window:i])
        
            X = standardized_returns
            y = standardized_market_index_returns
        
            model = get_linear_regression_model(X, y, fit_intercept=False)
            beta = model.coef_[0][0]

            if abs(beta) >= 1:
                raise Exception(f"Beta value {beta} is not in the range [-1, 1]")
            
            temp.append(beta)

        betas_corr[i] = temp

    return betas_corr 



def get_market_index_returns():
    market_index_returns = []

    for i in range(0, test_end, forecast_horizon):
        # We assume equal volatility across all stocks, simplifying our model to equal weights.
        volatility = {stock: 1 for stock in all_stocks}

        # Calculate portfolio weights based on volatility
        weights = get_portfolio_weights(volatility)

        # Calculate portfolio returns for each day in the window
        returns = np.dot(returns_matrix[i:i+forecast_horizon], weights)
        
        # Record portfolio returns
        market_index_returns.extend(returns)

    return market_index_returns



def get_training_data():
    X_train = []
    Y_train = []

    betas_corr = {}

    # Iterate over each window of returns
    for i in range(train_start, train_end, forecast_horizon):
        for stock in training_stocks:

            std = []
            mean_ret = []
            mean_vix = []
            betas = []
            temp = []

            for window in windows:

                #dobij sve feature koje ubacujemo u model
                std.append(returns_df[stock][i - window:i].std()* (252 ** 0.5) * 100)
                mean_ret.append(returns_df[stock][i - window:i].mean() * 252)
                mean_vix.append(float(vix_data[i - window: i].mean().iloc[0]))

                standardized_returns = standardize_returns(returns_df[stock][i - window:i])
                #print(methods["EqualWeight"]["returns"][i - window:i])

                standardized_market_index_returns = standardize_returns(market_index_returns[i - window:i])
            
                X = standardized_returns
                y = standardized_market_index_returns
            
                model = get_linear_regression_model(X, y, fit_intercept=False)
                beta = model.coef_[0][0]
                if abs(beta) >= 1:
                    raise Exception(f"Beta value {beta} is not in the range [-1, 1]")
                betas.append(beta)

            temp.append(beta)

            # Grupiraj kvadrirane povrate i prosječni VIX u jednu listu prije dodavanja u X_train
            features = std + mean_ret + mean_vix + betas
            
            # Sada dodaj tu listu kao jedan element u A
            X_train.append(features)
            Y_train.append(returns_df[stock][i:i + forecast_horizon].std())

        betas_corr[i] = temp

    return np.array(X_train), np.array(Y_train)


def test_model(model):
    volatility = {}

    for i in range(test_start, test_end, forecast_horizon):

        betas_for_corr = []
        temp = []
        
        for stock in all_stocks:

            std = []
            mean_ret = []
            mean_vix = []
            betas = []

            for window in windows:
                #dobij sve feature koje ubacujemo u model
                std.append(returns_df[stock][i - window:i].std() * (252 ** 0.5) * 100)
                mean_ret.append(returns_df[stock][i - window:i].mean() * 252)
                mean_vix.append(float(vix_data[i - window: i].mean().iloc[0]))

                standardized_returns = standardize_returns(returns_df[stock][i - window:i])
                standardized_market_index_returns = standardize_returns(market_index_returns[i - window:i])
            

                X = standardized_returns
                y = standardized_market_index_returns
                
                model = get_linear_regression_model(X, y, fit_intercept=False)
                beta = model.coef_[0][0]
                if abs(beta) >= 1:
                    raise Exception(f"Beta value {beta} is not in the range [-1, 1]")
                betas.append(beta)

                if window == 126:
                    betas_for_corr.append(beta)

            # Grupiraj kvadrirane povrate i prosječni VIX u jednu listu prije dodavanja u A
            features = std + mean_ret + mean_vix + betas
            
            features = np.array(features).reshape(1, -1)
            #  Trying to predict the volatility of the next forecast_horizon days
            volatility[stock] = methods[method]["model"].predict(features).item()
            print(type(volatility[stock]))
            temp.append(volatility[stock])
        
        R = predict_corr(betas_for_corr)
        V = np.diag(temp)
        S = np.dot(np.dot(V, R), V)

        ef_cmv = EfficientFrontier(None, S, weight_bounds=(0, 1))
        ef_gmv = EfficientFrontier(None, S, weight_bounds=(-1, 1))
        ef_cmv.min_volatility()
        ef_gmv.min_volatility()

        weights_cmv = ef_cmv.clean_weights()
        weights_gmv = ef_gmv.clean_weights()

        gather_portfolio_metrics(method + "cmv", i, weights_cmv, volatility)
        gather_portfolio_metrics(method + "gmv", i, weights_gmv, volatility)

In [7]:
market_index_returns = get_market_index_returns()

betas_corr = get_betas()

Training data

In [8]:
X_train, Y_train = get_training_data()

Benchmark methods

In [ ]:
method = "Oracle" + "+" + str(forecast_horizon)
reset_portfolio_metrics(method)


for i in range(test_start, test_end - 21, forecast_horizon):

    volatility = {}
    temp = []

    for stock in all_stocks:
        volatility[stock] = returns_df[stock][i: i + forecast_horizon].std()
        temp.append(volatility[stock])
    
    R = predict_corr(betas_corr[i + 21])
    V = np.diag(temp)
    S = np.dot(np.dot(V, R), V)

    ef_cmv = EfficientFrontier(None, S, weight_bounds=(0, 1))
    ef_gmv = EfficientFrontier(None, S, weight_bounds=(-1, 1))
    ef_cmv.min_volatility()
    ef_gmv.min_volatility()

    weights_cmv = ef_cmv.clean_weights()
    weights_gmv = ef_gmv.clean_weights()

    gather_portfolio_metrics(method + "cmv", i, weights_cmv, volatility)
    gather_portfolio_metrics(method + "gmv", i, weights_gmv, volatility)


method = "EqualWeight"
reset_portfolio_metrics(method)

for i in range(test_start, test_end, forecast_horizon):
    
    volatility = {}
    temp = []

    for stock in all_stocks:
        volatility[stock] = returns_df[stock][i: i + forecast_horizon].std()
        temp.append(volatility[stock])
    
    R = np.eye(len(all_stocks))
    V = np.diag(temp)
    S = np.dot(np.dot(V, R), V)

    ef_cmv = EfficientFrontier(None, S, weight_bounds=(0, 1))
    ef_gmv = EfficientFrontier(None, S, weight_bounds=(-1, 1))
    ef_cmv.min_volatility()
    ef_gmv.min_volatility()

    weights_cmv = ef_cmv.clean_weights()
    weights_gmv = ef_gmv.clean_weights()

    gather_portfolio_metrics(method + "cmv", i, weights_cmv, volatility)
    gather_portfolio_metrics(method + "gmv", i, weights_gmv, volatility)

Linear regression

In [10]:
def get_linear_regression_model(A, B, fit_intercept=True):
    # Fit a linear regression model with non-negative coefficients and no intercept
    model = LinearRegression(positive=True, fit_intercept=fit_intercept)
    model.fit(A, B)

    return model

In [ ]:
method = "LinReg(features,std)" + "-" + str(forecast_horizon)

methods[method]["model"] = get_linear_regression_model(X_train, Y_train, False) 
# Intercept might make variance negative

plt.figure(figsize=(25, 6))
plt.plot(methods[method]["model"].coef_)
plt.title(method + " Weights")
plt.xticks(range(len(x_label)), x_label)
plt.show()

test_model(method)


In [36]:
#potroflio metrics
portfolios_df = {
    "Portfolio": [method for method in methods],
    "Volatility": [np.std(portfolio["returns"]) * 252**0.5 \
                   for portfolio in methods.values()],
    "Return": [(1 + np.average(portfolio["returns"]))**252 - 1 \
               for portfolio in methods.values()],
    "Sharpe": [((1 + np.average(portfolio["returns"]))**252 - 1) \
               / (np.std(portfolio["returns"]) * 252**0.5) \
                for portfolio in methods.values()]
}

display(pd.DataFrame(portfolios_df))

c:\Users\filip\miniconda3\envs\everything\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\filip\miniconda3\envs\everything\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\filip\miniconda3\envs\everything\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\filip\miniconda3\envs\everything\Lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\filip\miniconda3\envs\everything\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,Portfolio,Volatility,Return,Sharpe
0,Oracle+21,NaN,NaN,NaN
1,Oracle+21cmv,0.113429,0.162631,1.433764
2,Oracle+21gmv,0.114000,0.131402,1.152644
3,"LinReg(features,std)-21",NaN,NaN,NaN
4,"LinReg(features,std)-21cmv",0.128233,0.165628,1.291619
5,"LinReg(features,std)-21gmv",0.131702,0.128361,0.974633
6,EqualWeight,NaN,NaN,NaN
7,EqualWeightcmv,0.127901,0.231916,1.813237
8,EqualWeightgmv,0.127901,0.231916,1.813237
